In [5]:
#import sys
#import pathlib
#src_path = pathlib.Path().absolute().parent / "src"
#sys.path.append(str(src_path))
#import data_functions

import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
#from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline 
from collections import Counter
from sklearn.datasets import make_classification

import sklearn

import pickle

from sklearn.metrics import plot_confusion_matrix, confusion_matrix, classification_report

from sklearn.datasets import make_blobs
from sklearn.datasets import make_moons

 
import numpy as np

from sklearn.svm import SVC


#import model features pipeline
strict_pp = src_path / 'strict_pre_pipeline_v4.pkl'

In [20]:

import sys
import pathlib
src_path = pathlib.Path().absolute().parent / "src"
sys.path.append(str(src_path))
import data_functions
import custom_transformers as ct
import pandas as pd
import numpy as np

import pickle

from sklearn.compose import  make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

In [6]:
x_train, x_test, y_train = data_functions.get_dataframes()

In [18]:
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

import data_functions
import numpy as np

from datetime import datetime

class BinInstaller(TransformerMixin, BaseEstimator):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def _to_df(self, X):
        if type(X) != pd.DataFrame:
            if type(X) != list:
                if type(X) == pd.Series:
                    data = pd.DataFrame(X)
                elif type(X) == dict:
                    data = pd.DataFrame([X])
                else:
                    raise ValueError('X must be a dataframe, list, series, or dictionary  object.')
            else:
                data = pd.DataFrame(X)
        else:
            data = X.copy()
        return data
        
    def transform(self, X):
        data = self._to_df(X)
        others = data['installer'].value_counts().index[data['installer'].value_counts() < 10]
        is_other = lambda x: 'Other' if x in others else x
        data['installer'] = data['installer'].map(is_other)
        data['installer'] = data['installer'].fillna('Unknown')
        return data

    
class TransformConstructionYear(TransformerMixin, BaseEstimator):
    def __init__(self):
        self.current_year = datetime.now().year
    
    def fit(self, X, y=None):
        return self
    
    def _to_df(self, X):
        if type(X) != pd.DataFrame:
            if type(X) != list and type(X) != np.ndarray:
                if type(X) == pd.Series:
                    data = pd.DataFrame(X)
                elif type(X) == dict:
                    data = pd.DataFrame([X])
                else:
                    raise ValueError('X must be a dataframe, list, series, or dictionary  object.')
                    
            else:
                data = pd.DataFrame(X)
        else:
            data = X.copy()
        return data
    
    def _bin_data(self, x):
        if x == 0:
            return 0 
        else:
            return self.current_year - x 
        
    def transform(self, X):
        data = self._to_df(X)
        data['construction_year'] = data['construction_year'].apply(self._bin_data)
        return data
    
class ChooseStrictFeatures(TransformerMixin, BaseEstimator):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def _to_df(self, X):
        if type(X) != pd.DataFrame:
            if type(X) != list:
                if type(X) == pd.Series:
                    data = pd.DataFrame(X)
                elif type(X) == dict:
                    data = pd.DataFrame([X])
                else:
                    raise ValueError('X must be a dataframe, list, series, or dictionary  object.')
            else:
                data = pd.DataFrame(X)
        else:
            data = X.copy()
        return data
        
    def transform(self, X):
        data = self._to_df(X)
        data = data[data_functions.get_strict_features()]
        return data
    
# class ChooseLooseFeatures(TransformerMixin, BaseEstimator):
#     def __init__(self):
#         pass
    
#     def fit(self, X, y=None):
#         return self
    
#     def _to_df(self, X):
#         if type(X) != pd.DataFrame:
#             if type(X) != list:
#                 if type(X) == pd.Series:
#                     data = pd.DataFrame(X)
#                 elif type(X) == dict:
#                     data = pd.DataFrame([X])
#                 else:
#                     raise ValueError('X must be a dataframe, list, series, or dictionary  object.')
#             else:
#                 data = pd.DataFrame(X)
#         else:
#             data = X.copy()
#         return data
        
#     def transform(self, X):
#         data = self._to_df(X)
#         data = data[data_functions.get_loose_features()]
#         return data


In [26]:
strict_pp_pipe = Pipeline([('feature_selector', ChooseStrictFeatures()),
                           ('bin_installer', ct.BinInstaller()),
                           ('column_transformer', make_column_transformer((SimpleImputer(missing_values = 0.0), ['construction_year']),
                                                            (SimpleImputer(), data_functions.get_numeric_features(data_functions.get_strict_features())),
                                                            (ct.TransformConstructionYear(), ['construction_year']),
                                                            (StandardScaler(), data_functions.get_numeric_features(data_functions.get_strict_features())),
                                                            (OneHotEncoder(handle_unknown = 'ignore', sparse = False), data_functions.get_categorical_features(data_functions.get_strict_features())),
                                                            remainder = 'drop')),
                           ('up_sample', SMOTE(random_state = 42))])

In [27]:
resampled_data = strict_pp_pipe.fit_resample(x_train, y_train)

In [28]:
x_tr_resamp= pd.DataFrame(resampled_data[0])

In [29]:
strict_model = Pipeline([('preprocessor', resampled_data), ('dTree_classifier', DecisionTreeClassifier())])


TypeError: All intermediate steps of the chain should be estimators that implement fit and transform or fit_resample (but not both) or be a string 'passthrough' '(array([[ 1.99900000e+03,  6.00000000e+03,  1.39000000e+03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.01000000e+03,  0.00000000e+00,  1.39900000e+03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.00900000e+03,  2.50000000e+01,  6.86000000e+02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.97719360e+03,  0.00000000e+00,  8.27015993e+01, ...,
         0.00000000e+00,  0.00000000e+00,  2.98400650e-01],
       [ 1.98800000e+03,  0.00000000e+00, -3.50885787e+01, ...,
         5.65397631e-01,  0.00000000e+00,  4.34602369e-01],
       [ 1.99681469e+03,  0.00000000e+00,  0.00000000e+00, ...,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00]]),          status_group
0          functional
1          functional
2          fu

In [ ]:
cross_val_score(strict_model, x_train, y_train, cv = 5)


In [7]:
#file = open(strict_pp, 'rb')
#strict_v4 = pickle.load(file)
#file.close()

In [13]:
#strict_v4

In [14]:
#strict_v4.fit_transform(x_train)

In [10]:
#x_tr_val, x_te_val, y_tr_val, y_te_val = train_test_split(x_train, y_train)

In [11]:

#resampled_data = strict_v4.fit_resample(x_tr_val, y_tr_val)

In [12]:
#x_tr_resamp= pd.DataFrame(resampled_data[0])

In [15]:
#x_train.shape

In [ ]:
#x_tr_val, x_te_val, y_tr_val, y_te_val = train_test_split(x_train, y_train)

In [ ]:
#file = open(strict_pickle_path, 'rb')
#strict_v4 = pickle.load(file)
#file.close()
#x_tr_val, x_te_val, y_tr_val, y_te_val = train_test_split(x_train, y_train)
#resampled_data = strict_v4.fit_resample(x_tr_val, y_tr_val)
#x_tr_resamp= pd.DataFrame(resampled_data[0])
#y_tr_resamp = pd.DataFrame(resampled_data[1])

#x_tr_resamp= x_tr_resamp.drop(columns=[0,1,2,3,4,5], axis = 1)
